# Enable application insights and add custom logs in your endpoint

## Get your Azure ML Workspace

In [ ]:
!pip install azureml-core

In [ ]:
import azureml
from azureml.core import Workspace
import mlflow.azureml

workspace_name = '<YOUR-WORKSPACE-NAME>'
resource_group = '<YOUR-RESOURCE-GROUP>'
subscription_id = '<YOUR-SUBSCRIPTION-ID>'

workspace = Workspace.get(name = workspace_name,
                          resource_group = resource_group,
                          subscription_id = subscription_id)

## Customize your entry script to add some custom logs

In [ ]:
%%writefile /dbfs/models/churn-prediction/score.py

import mlflow
import json
import pandas as pd
import os
import xgboost as xgb
import time
import datetime

# Called when the deployed service starts
def init():
    global model
    global train_stats

    # Get the path where the deployed model can be found.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), './churn-prediction')
    
    # Load model
    model = mlflow.xgboost.load_model(model_path)

# Handle requests to the service
def run(rawdata):
  try:
    data = pd.read_json(rawdata, orient = 'split')
    data_xgb = xgb.DMatrix(data) 
    
    start_time = datetime.datetime.now()
    # Return the prediction
    prediction = predict(data_xgb)
    end_time = datetime.datetime.now()
    
    print(f'TOTAL_TIME (ms): {end_time - start_time}') # TRACK IN APP INSIGHTS
    
    info = json.dumps({"payload": rawdata})
    print(f'OUR_PAYLOAD: {info}') # TRACK IN APP INSIGHTS
    
    return prediction
    
  except Exception as e:
    error = str(e)
    print (f'ERROR: {error + time.strftime("%H:%M:%S")}') # TRACK IN APP INSIGHTS
    raise Exception(error)

def predict(data):
  prediction = model.predict(data)[0]
  return {"churn-prediction": str(prediction)}

## Define your inference config (same we already did)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = Environment(name='xgboost_env')

conda_dep = CondaDependencies('/dbfs/models/churn-prediction/conda.yaml')

# Define the packages needed by the model and scripts
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="/dbfs/models/churn-prediction/score.py",
                                   environment=env)

## Get the registered model

In [ ]:
from azureml.core.model import Model

model_name = 'churn-model'
model_azure = Model.list(workspace = workspace, name = model_name)[0]

## New deployment on AKS. Now with App insights enable

In [ ]:
from azureml.core.webservice import AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException

endpoint_name = 'api-churn-prod'
aks_name = 'aks-e2e-ds'

try:
  aks_target = AksCompute(workspace, aks_name)
except ComputeTargetException as error:
  prov_config = AksCompute.provisioning_configuration()
  aks_target = ComputeTarget.create(workspace = workspace, name = aks_name, provisioning_configuration = prov_config)
  aks_target.wait_for_completion(show_output = True)

aks_config = AksWebservice.deploy_configuration(enable_app_insights = True)
aks_service = Model.deploy(workspace=workspace,
                           name=endpoint_name,
                           models=[model_azure],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Call the API and see the results in the `Application Insights`

Let's send 10 requests.

In [ ]:
import requests

payload1='{"columns":["Idade","RendaMensal","PercentualUtilizacaoLimite","QtdTransacoesNegadas","AnosDeRelacionamentoBanco","JaUsouChequeEspecial","QtdEmprestimos","NumeroAtendimentos","TMA","IndiceSatisfacao","Saldo","CLTV"],"data":[[21,9703,1.0,5.0,12.0,0.0,1.0,100,300,2,6438,71]]}'

payload2='{"columns":["Idade","RendaMensal","PercentualUtilizacaoLimite","QtdTransacoesNegadas","AnosDeRelacionamentoBanco","JaUsouChequeEspecial","QtdEmprestimos","NumeroAtendimentos","TMA","IndiceSatisfacao","Saldo","CLTV"],"data":[[21,9703,1.0,5.0,12.0,0.0,1.0,1,5,5,6438,71]]}'

headers = {
  'Content-Type': 'application/json'
}

prod_service_key = aks_service.get_keys()[0] if len(aks_service.get_keys()) > 0 else None

headers["Authorization"] = "Bearer {service_key}".format(service_key=prod_service_key)

for count in range(5):
  print(f'Predição: {count}')
  response1 = requests.request("POST", aks_service.scoring_uri, headers=headers, data=payload1)
  response2 = requests.request("POST", aks_service.scoring_uri, headers=headers, data=payload2)

  print(response1.text)
  print(response2.text)

Go to ***Azure Portal***, access ***Appication Insights*** and see the chart "Server Requests" at the ***Overview*** page. 

![Application Insights Overview](media/1-appins-overview.png)

To see all requests go to ***Investigate > Transaction search***.

![Application Insights End-to-end transaction details](media/2-appins-transactionsearch.PNG)


Click on one of the requests to see more details about it.

![Application Insights End-to-end transaction details](media/3-appins-endtoend-transaction.PNG)


### Let's try to simulate some errors as well.

In [ ]:
payload3='{"columns":["Idade","RendaMensalERRO","PercentualUtilizacaoLimite","QtdTransacoesNegadas","AnosDeRelacionamentoBanco","JaUsouChequeEspecial","QtdEmprestimos","NumeroAtendimentos","TMA","IndiceSatisfacao","Saldo","CLTV"],"data":[[21,9703,1.0,5.0,12.0,0.0,1.0,1,5,5,6438,71]]}'

for count in range(10):
  response1 = requests.request("POST", aks_service.scoring_uri, headers=headers, data=payload3)
  print(response1.text)
  print('\n')

Go to ***Azure Portal***, access ***Appication Insights*** and see the chart "Failed Requests" at the ***Overview*** page. 

![Application Insights Overview](media/4-appins-overview-failed.png)

To see all failed requests go to ***Investigate > Transaction search***.

![Application Insights end-to-end trasaction details](media/5-appins-transactionsearch-failed.PNG)

Click on one of the failed requests to see more details about it.

![Application Insights End-to-end transaction details](media/6-appins-endtoend-transaction-failed.PNG)


## Query logs in Application Insights

To analyze captured log data by Application Insights, you can use the Log Analytics query interface in the Azure portal. This interface supports a SQL-like query syntax that you can use to extract fields from logged data, including custom dimensions created by your Azure Machine Learning service.

Access ***Appication Insights***, go to ***Monitoring > logs*** and paste the query below to start analyzing your logs on Application Insights:

``` sql
traces
|where customDimensions.["Service Name"] == "api-churn-prod"
| project  timestamp, customDimensions.Content, message, client_StateOrProvince
```

![Application Insights Logs Query](media/7-appins-query.PNG)

## Update your endpoint to enable/disable `application insights`

You can enable the `Application Insights` on an existing endpoint as well, to do this just run the code bellow passing True/False value.

### Get your endpoint (the ACI/AKS endpoint your already deployed)

In [ ]:
from azureml.core.webservice import Webservice

endpoint_name = 'api-churn-prod'
aks_service= Webservice(workspace, endpoint_name)

### Enable or disable `Application Insights`

In [ ]:
aks_service.update(enable_app_insights=True)